In [46]:
import logging
import datetime
import json

def get_api(url):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.get(url)
    return response

def post_api(url, data):
    import google.auth
    credentials, project = google.auth.default()
    from google.auth.transport.requests import AuthorizedSession
    authed_session = AuthorizedSession(credentials)
    response = authed_session.post(url, data)
    return response

def get_createdby(createTime, jobId):
    createdBy=""
    import arrow
    createTime = arrow.get(createTime)
    logstart=createTime + datetime.timedelta(minutes = -2)
    logend=createTime + datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    y = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', y)
    import pandas as pd
    data_df = pd.read_json(response.text)
    if 'entries' in data_df:
        for job in data_df['entries']:
            if job["protoPayload"]["request"]["job"]["jobId"] == jobId:
                createdBy=job["protoPayload"]["authenticationInfo"]["principalEmail"]
                break
    else:
        if response.status_code == 429:
            import time
            # print("Sleeping 1 Sec: " + jobId)
            time.sleep(1)
            createdBy=get_createdby(createTime, jobId)
            #createdBy="Error"
            
    return createdBy
    
def get_trainingcost(consumedMLUnits):
    trainingCost=""
    import arrow
    trainingCost = arrow.get(trainingOutput.consumedMLUnits)
    logstart=datetime.timedelta(minutes = -2)
    logend=datetime.timedelta(minutes = 2)
    body = {"resourceNames": ["projects/sb-bigdata-4985-da852265"], "filter":'resource.type="audited_resource" \
    resource.labels.service="ml.googleapis.com" \
    resource.labels.method="google.cloud.ml.v1.JobService.CreateJob" \
    timestamp>="' + str(logstart) + '" \
    timestamp<="' + str(logend) + '"'
    }
    x = json.dumps(body)
    response=post_api('https://logging.googleapis.com/v2/entries:list', x)
    import pandas as pd
    data_df = pd.read_json(response.text)
    for job in data_df['entries']:
        trainingCost = trainingCost*0.49
            
    return trainingCost   
    
def get_jobs_df():
    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs')
    job_list=[]
    if response.status_code == 200:
        while True:
            import json
            import pandas as pd
            data_df = pd.read_json(response.text)
            for job in data_df['jobs']:
                job["createdby"]=get_createdby(job["createTime"], job["jobId"])
                #job["trainingcost"]=get_trainingcost(job["consumedMLUnits"])
                job_list.append(job)
                
            page_token = None
            page_token = data_df.get('nextPageToken')
            if page_token is None:
                break
            else:
                response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs' + '?pageToken=' + str(page_token[0]))
    from pandas.io.json import json_normalize
    job_df=json_normalize(job_list)
    job_df_new = job_df.rename(columns={"createTime": "createtime", "endTime": "endtime", "jobId":"jobid", "startTime":"starttime"
                       , "trainingInput.args":"args", "trainingInput.jobDir":"jobdir", "trainingInput.masterConfig.imageUri":"imageuri"
                       , "trainingInput.masterType":"mastertype", "trainingInput.packageUris":"packageuris", "trainingInput.pythonModule":"pythonmodule"
                       , "trainingInput.pythonVersion":"pythonversion", "trainingInput.region":"region", "trainingInput.runtimeVersion":"runtimeversion"
                       , "trainingInput.scaleTier":"scaletier", "trainingOutput.consumedMLUnits":"consumedmlunits", "trainingCost":"trainingcost"})
    return job_df_new

def write_to_bqtable(bq_tablename, bq_schemaname, datadf):
    import google.auth
    from google.cloud import bigquery
    credentials, project = google.auth.default()
    client = bigquery.Client(project)
    job_config = bigquery.LoadJobConfig()
    job_config.schema = bq_schemaname
    
    job = client.load_table_from_dataframe(datadf, bq_tablename, job_config=job_config)
    # Wait for the load job to complete.
    job.result()

def get_data(request): 
    jobs = []
    jobs = get_jobs_df()
    jobs = jobs.drop(columns=["errorMessage", "etag"])
    print("AI platform jobs count : ",str(len(jobs)))
    jobs.insert(0, 'date', str(datetime.datetime.utcnow()))
    from google.cloud import bigquery
    varbqschema=[
        bigquery.SchemaField(name="date", field_type="STRING"),
        bigquery.SchemaField(name="createtime", field_type="STRING"),
        bigquery.SchemaField(name="createdby", field_type="STRING"),
        bigquery.SchemaField(name="endtime", field_type="STRING"),
        bigquery.SchemaField(name="jobid", field_type="STRING"),
        bigquery.SchemaField(name="starttime", field_type="STRING"),
        bigquery.SchemaField(name="state", field_type="STRING"),
        bigquery.SchemaField(name="args", field_type="STRING"),
        bigquery.SchemaField(name="jobdir", field_type="STRING"),
        bigquery.SchemaField(name="imageuri", field_type="STRING"),
        bigquery.SchemaField(name="mastertype", field_type="STRING"),
        bigquery.SchemaField(name="packageuris", field_type="STRING"),
        bigquery.SchemaField(name="pythonmodule", field_type="STRING"),
        bigquery.SchemaField(name="pythonversion", field_type="STRING"),
        bigquery.SchemaField(name="region", field_type="STRING"),
        bigquery.SchemaField(name="runtimeversion", field_type="STRING"),
        bigquery.SchemaField(name="scaletier", field_type="STRING"),
        bigquery.SchemaField(name="consumedmlunits", field_type="STRING"),
        bigquery.SchemaField(name="trainingcost", field_type="STRING")
    ]
    write_to_bqtable('modelmanagement.jobstemp', varbqschema, jobs.astype(str))
    return "Done!"

In [47]:
print(datetime.datetime.now())
get_data(None)
print(datetime.datetime.now())

2019-12-09 14:07:53.942324
AI platform jobs count :  827


ValueError: bq_schema contains fields not present in dataframe: {'trainingcost'}

In [6]:
def get_jobs_df():
    response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs')
    job_list=[]
    if response.status_code == 200:
        while True:
            import json
            import pandas as pd
            data_df = pd.read_json(response.text)
            for job in data_df['jobs']:
                job["createdby"]=get_createdby(job["createTime"], job["jobId"])
                job_list.append(job)
                
            page_token = None
            page_token = data_df.get('nextPageToken')
            if page_token is None:
                break
            else:
                response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs' + '?pageToken=' + str(page_token[0]))
    from pandas.io.json import json_normalize
    job_df=json_normalize(job_list)
    job_df_new = job_df.rename(columns={"createTime": "createtime", "endTime": "endtime", "jobId":"jobid", "startTime":"starttime"
                       , "trainingInput.args":"args", "trainingInput.jobDir":"jobdir", "trainingInput.masterConfig.imageUri":"imageuri"
                       , "trainingInput.masterType":"mastertype", "trainingInput.packageUris":"packageuris", "trainingInput.pythonModule":"pythonmodule"
                       , "trainingInput.pythonVersion":"pythonversion", "trainingInput.region":"region", "trainingInput.runtimeVersion":"runtimeversion"
                       , "trainingInput.scaleTier":"scaletier", "trainingOutput.consumedMLUnits":"consumedmlunits"})
    return job_df_new

In [7]:
response = get_api('https://ml.googleapis.com/v1/projects/sb-bigdata-4985-da852265/jobs')

In [16]:
import arrow
trainingCost = arrow.get(consumedMLUnits)

NameError: name 'consumedMLUnits' is not defined